In [2]:
import cv2
import torch
from pathlib import Path
import time
def adjust_contrast(image, alpha, beta):
    """
    Adjust the contrast of the image using the formula: new_image = alpha * image + beta
    """
    adjusted_image = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)
    return adjusted_image
# Load YOLOv5 model
model = torch.hub.load('ultralytics/yolov5','custom', path='Yolov5_StrongSORT_OSNet/yolov5/runs/train/exp16/weights/best.pt', force_reload=True)
model.eval()

# Open video file
video_path = 'validate05.mp4'
cap = cv2.VideoCapture(video_path)
# Get video properties
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
alpha = 2  # Increase this value to increase contrast
beta = 2     # Increase this value to make the image brighter or decrease to make it darker
# Define output video writer
output_path = 'M-1-5.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')

frame_number = 0
total_false_positives = 0
start_time = time.time()
frame_count = 0
new_width = 1920  # New width in pixels
new_height = 1080  # New height in pixels
out = cv2.VideoWriter(output_path, fourcc, fps, (new_width, new_height))
while cap.isOpened():
    # Read a frame
    ret, frame = cap.read()
    if not ret:
        break
    frame = cv2.resize(frame, (new_width, new_height))
    frame= adjust_contrast(frame, alpha, beta)
    # Convert frame to RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Perform object detection
    results = model(frame_rgb)
    image_height, image_width = frame.shape[:2]
    r1=0.66
    r2=1.47
    frame_count += 1
    # Process detection results
    if results.pred[0] is not None:
        for *xyxy, conf, cls in results.xyxy[0]:
            if conf>0.4:
                x1, y1, x2, y2 = [int(i) for i in xyxy]
                w = x2 - x1
                h = y2 - y1
                if w==0 or h==0:
                    r=0
                else:
                    r=w/h
                if w <100 and h<100 and r1<r<r2:
                    total_false_positives += len(results.pred[0])
                    cv2.putText(frame, "WARNING!", (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 3)
                    frame_with_notification = draw_notification(frame)
                    save_predictions(results, frame_number, image_width, image_height)
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(frame, f'drone {conf:.2f}', (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    end_time = time.time()
    elapsed_time = end_time - start_time
    fps = frame_count / elapsed_time
    print(f"Average FPS: {fps:.2f}")
    # Write annotated frame to output video
    frame_number += 1
    out.write(frame)

    # Display the frame
    cv2.imshow('Object Detection', cv2.resize(frame,None,fx=1,fy=1))
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()
print(f'Total False Positives: {total_false_positives}')

C:\Users\admin\anaconda3\envs\pytorch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\admin/.cache\torch\hub\master.zip
YOLOv5  2023-8-19 Python-3.7.16 torch-1.13.1+cu117 CUDA:0 (NVIDIA GeForce GTX 1650, 4096MiB)

Fusing layers... 
Model summary: 267 layers, 46108278 parameters, 0 gradients, 107.6 GFLOPs
Adding AutoShape... 


Average FPS: 7.25
Average FPS: 7.35
Average FPS: 7.56
Average FPS: 7.77
Average FPS: 7.97
Average FPS: 8.19
Average FPS: 8.45
Average FPS: 8.69
Average FPS: 8.89
Average FPS: 9.01
Average FPS: 9.14
Average FPS: 9.27
Average FPS: 9.33
Average FPS: 9.36
Average FPS: 9.40
Average FPS: 9.47
Average FPS: 9.48
Average FPS: 9.50
Average FPS: 9.55
Average FPS: 9.60
Average FPS: 9.63
Average FPS: 9.66
Average FPS: 9.68
Average FPS: 9.71
Average FPS: 9.74
Average FPS: 9.77
Average FPS: 9.79
Average FPS: 9.81
Average FPS: 9.83
Average FPS: 9.86
Average FPS: 9.88
Average FPS: 9.88
Average FPS: 9.90
Average FPS: 9.90
Average FPS: 9.90
Average FPS: 9.92
Average FPS: 9.86
Average FPS: 9.87
Average FPS: 9.88
Average FPS: 9.86
Average FPS: 9.87
Average FPS: 9.89
Average FPS: 9.92
Average FPS: 9.92
Average FPS: 9.93
Average FPS: 9.93
Average FPS: 9.94
Average FPS: 9.94
Average FPS: 9.96
Average FPS: 9.98
Average FPS: 10.00
Average FPS: 10.01
Average FPS: 10.01
Average FPS: 10.01
Average FPS: 10.01
Avera

In [1]:
def save_predictions(results, frame_number, image_width, image_height):
    # Create a directory to save the text files if it doesn't exist
    save_dir = 'P-1-5'
    Path(save_dir).mkdir(parents=True, exist_ok=True)

    # Get the predictions for each bounding box
    predictions = results.pandas().xyxy[0]

    # Convert normalized coordinates to absolute pixel values
    predictions['x'] = (predictions['xmin'] + predictions['xmax']) / (2 * image_width)
    predictions['y'] = (predictions['ymin'] + predictions['ymax']) / (2 * image_height)
    predictions['width'] = (predictions['xmax'] - predictions['xmin']) / image_width
    predictions['height'] = (predictions['ymax'] - predictions['ymin']) / image_height

    # Save the predictions to a text file in YOLOv5 format
    save_path = f'{save_dir}/frame_{frame_number}.txt'
    with open(save_path, 'w') as f:
        for _, row in predictions.iterrows():
            class_id = row['class']
            x = row['x']
            y = row['y']
            width = row['width']
            height = row['height']
            confidence = row['confidence']
            line = f"{class_id} {x} {y} {width} {height}\n"
            f.write(line)
def draw_notification(frame):
    height, width, _ = frame.shape
    square_size = min(height, width) // 10
    cv2.rectangle(frame, (width - square_size, 0), (width, square_size), (0, 0, 255), -1)
    return frame